<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>

<br>

# <font color="#76b900">**Notebook 8 [Assessment]:** RAG Evaluation</font>

<br>

Welcome to the last notebook of the course! In the previous notebook, you integrated a vector store solution into a RAG pipeline! In this notebook, you will take that same pipeline and evaluate it using numerical RAG evaluation techniques incorporating LLM-as-a-Judge metrics!

<br>

### **Learning Objectives:**

- Learn how to integrate the techniques from prior notebooks to numerically approximate the goodness of your RAG pipeline.

- **Final Exercice**: ***By working through this notebook in the Course Environment,* you will be able to submit the coding component of the course!**

<br>

### **Questions To Think About:**

- As you go along, remember what our metrics actually represent. Should our pipeline pass these objectives? Is our judge LLM sufficient for evaluating the pipeline? Does a particular metric even matter for our use case?
- If we left the vectorstore-as-a-memory component in our chain, do you think it would still pass the evaluation? Additionally, is the evaluation useful for assessing vectorstore-as-a-memory performance? 

<br>

### **Environment Setup:**

In [1]:
# %pip install -q langchain langchain-nvidia-ai-endpoints gradio rich
# %pip install -q arxiv pymupdf faiss-cpu ragas

## If you encounter a typing-extensions issue, restart your runtime and try again
# from langchain_nvidia_ai_endpoints import ChatNVIDIA
# ChatNVIDIA.get_available_models()

from functools import partial
from rich.console import Console
from rich.style import Style
from rich.theme import Theme

console = Console()
base_style = Style(color="#76B900", bold=True)
norm_style = Style(bold=True)
pprint = partial(console.print, style=base_style)
pprint2 = partial(console.print, style=norm_style)

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models(base_url="http://llm_client:9000/v1")
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")

----

<br>

## **Part 1:** Pre-Release Evaluation

In our previous notebook, we successfully combined several concepts to create a document chatbot with the aim of responsive and informative interactions. However, the diversity of user interactions necessitates comprehensive testing to truly understand the chatbot's performance. Thorough testing in varied scenarios is crucial to ensure that the system is not only robust and versatile but also aligns with user and provider expectations.

After defining your chatbot's roles and implementing the necessary features, evaluating it becomes a multi-stage process:

- **Typical Use Inspection:** Start by testing scenarios most relevant to your use case. See if your chatbot can reliably navigate discussions with limited human intervention.

    - Additionally, identify limitations or compartments that should be redirected to a human for inspection/supervision (i.e., human swap-in to confirm transactions or perform sensitive navigation) and implement those options.

- **Edge Case Inspection:** Explore the boundaries of typical use, identifying how the chatbot handles less common but plausible scenarios.

    - Before any public release, assess critical boundary conditions that could pose liability risks, such as the potential generation of inappropriate content.

    - Implement well-tested guardrails on all outputs (and possibly inputs) to limit undesired interactions and redirect users into predictable conversation flows.

- **Progressive Rollout:** Rolling out your model to a limited audience (first internal, then [A/B](https://en.wikipedia.org/wiki/A/B_testing)) and implement analytics features like usage analytics dashboards and feedback avenues (flag/like/dislike/etc).

Of these three steps, the first two can be done by a small team or an individual and should be iterated on as part of the development process. Unfortunately, this needs to be done frequently and can be prone to human error. **Luckily for us, LLMs can be used to help out with LLM-as-a-Judge formulations!**

*(Yeah, this probably isn't surprising by now. LLMs being strong is why this course is here...).*

----

<br>

## **Part 2:** LLM-as-a-Judge Formulation

In the realm of conversational AI, using LLMs as evaluators or 'judges' has emerged as a useful approach for configurable automatic testing of natural language task performance:

- An LLM can simulate a range of interaction scenarios and generate synthetic data, allowing an evaluation developer to generate targeted inputs to eliciting a range of behaviors from your chatbot.

- The chatbot's correspondence/retrieval on the synthetic data can be evaluated or parsed by an LLM and a consistent output format such as "Pass"/"Fail", similarity, or extraction can be enforced.

- Many such results can be aggregated and a metric can be derived which explains something like "% of passing evaluations", "average number of relevant details from the sources", "average cosine similarity", etc.

This idea of using LLMs to test out and quantify chatbot quality, known as [**"LLM-as-a-Judge,"**](https://arxiv.org/abs/2306.05685) allows for easy test specifications that align closely with human judgment and can be fine-tuned and replicated at scale.

**There are several popular frameworks for off-the-shelf judge formulations including:**
- [**RAGAs (short for RAG Assessment)**](https://docs.ragas.io/en/stable/), which offers a suite of great starting points for your own evaluation efforts.
- [**LangChain Evaluators**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/), which are similar first-party options with many implicitly-constructible agents.

Instead of using the chains as-is, we will instead expand on the ideas and evaluate our system with a more custom solution.

----

<br>

## **Part 3: [Assessment Prep]** Pairwise Evaluator

The following exercise will flesh out a custom implementation of a simplified [LangChain Pairwise String Evaluator](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/comparison/pairwise_string/). 

**To prepare for our RAG chain evaluation, we will need to:**

- Pull in our document index (the one we saved in the previous notebook).
- Recreate our RAG pipeline of choice.

**We will specifically be implementing a judge formulation with the following steps:**

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

**The chain should be a simple but powerful process that tests for the following objective:**

> ***Does my RAG chain outperform a narrow chatbot with limited document access.***





**This will be the system used for the final evaluation!** To see how this system is integrated into the autograder, please check out the implementation in [`frontend/server_app.py`](frontend/server_app.py).

<br>

### **Task 1:** Pull In Your Document Retrieval Index

For this exercise, you will pull in the `docstore_index` file you created as part of your earlier notebook. The following cell should be able to load in the store as-is.

In [2]:
## Make sure you have docstore_index.tgz in your working directory
from langchain_nvidia_ai_endpoints import NVIDIAEmbeddings
from langchain_community.vectorstores import FAISS

# embedder = NVIDIAEmbeddings(model="nvidia/embed-qa-4", truncate="END")

!tar xzvf docstore_index.tgz
docstore = FAISS.load_local("docstore_index", embedder, allow_dangerous_deserialization=True)
docs = list(docstore.docstore._dict.values())

def format_chunk(doc):
    return (
        f"Paper: {doc.metadata.get('Title', 'unknown')}"
        f"\n\nSummary: {doc.metadata.get('Summary', 'unknown')}"
        f"\n\nPage Body: {doc.page_content}"
    )

## This printout just confirms that your store has been retrieved
pprint(f"Constructed aggregate docstore with {len(docstore.docstore._dict)} chunks")
pprint(f"Sample Chunk:")
print(format_chunk(docs[len(docs)//2]))

docstore_index/
docstore_index/index.pkl
docstore_index/index.faiss


Constructed aggregate docstore with 918 chunks

Sample Chunk:

Paper: Pandora: Jailbreak GPTs by Retrieval Augmented Generation Poisoning

Summary: Large Language Models~(LLMs) have gained immense popularity and are being
increasingly applied in various domains. Consequently, ensuring the security of
these models is of paramount importance. Jailbreak attacks, which manipulate
LLMs to generate malicious content, are recognized as a significant
vulnerability. While existing research has predominantly focused on direct
jailbreak attacks on LLMs, there has been limited exploration of indirect
methods. The integration of various plugins into LLMs, notably Retrieval
Augmented Generation~(RAG), which enables LLMs to incorporate external
knowledge bases into their response generation such as GPTs, introduces new
avenues for indirect jailbreak attacks.
  To fill this gap, we investigate indirect jailbreak attacks on LLMs,
particularly GPTs, introducing a novel attack vector named Retrieval Augmented
Generation Poisoning. This method, Pandora, exploits the 

<br>

### **Task 2: [Exercise]** Pull In Your RAG Chain

Now that we have our index, we can recreate the RAG agent from the previous notebook! 

**Key Modifications:**
- To keep things simple, feel free to disregard the vectorstore-as-a-memory component. Incorporating it will require some more overhead and will make the exercise a bit more complicated.

In [5]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableBranch
from langchain_core.runnables.passthrough import RunnableAssign
from langchain.document_transformers import LongContextReorder

from langchain_nvidia_ai_endpoints import ChatNVIDIA, NVIDIAEmbeddings

from functools import partial
from operator import itemgetter

import gradio as gr

#####################################################################

# NVIDIAEmbeddings.get_available_models()
embedder = NVIDIAEmbeddings(model="nvidia/nv-embed-v1", truncate="END")

# ChatNVIDIA.get_available_models()
instruct_llm = ChatNVIDIA(model="meta/llama-3.1-8b-instruct")
llm = instruct_llm | StrOutputParser()

#####################################################################

def docs2str(docs, title="Document"):
    """Useful utility for making chunks into context string. Optional, but useful"""
    out_str = ""
    for doc in docs:
        doc_name = getattr(doc, 'metadata', {}).get('Title', title)
        if doc_name: out_str += f"[Quote from {doc_name}] "
        out_str += getattr(doc, 'page_content', str(doc)) + "\n"
    return out_str

chat_prompt = ChatPromptTemplate.from_template(
    "You are a document chatbot. Help the user as they ask questions about documents."
    " User messaged just asked you a question: {input}\n\n"
    " The following information may be useful for your response: "
    " Document Retrieval:\n{context}\n\n"
    " (Answer only from retrieval. Only cite sources that are used. Make your response conversational)"
    "\n\nUser Question: {input}"
)

def output_puller(inputs):
    """"Output generator. Useful if your chain returns a dictionary with key 'output'"""
    if isinstance(inputs, dict):
        inputs = [inputs]
    for token in inputs:
        if token.get('output'):
            yield token.get('output')

#####################################################################
## TODO: Pull in your desired RAG Chain. Memory not necessary

## Chain 1 Specs: "Hello World" -> retrieval_chain 
##   -> {'input': <str>, 'context' : <str>}
long_reorder = RunnableLambda(LongContextReorder().transform_documents)  ## GIVEN
context_getter = itemgetter('input') | docstore.as_retriever() | long_reorder | docs2str
#context_getter = RunnableLambda(lambda x: x)## TODO
retrieval_chain = {'input' : (lambda x: x)} | RunnableAssign({'context' : context_getter})

## Chain 2 Specs: retrieval_chain -> generator_chain 
##   -> {"output" : <str>, ...} -> output_puller
#generator_chain = RunnableAssign({"output" : chat_prompt | llm })  ## TODO
#generator_chain = generator_chain | output_puller  ## GIVEN

generator_chain = chat_prompt | llm   ## TODO
generator_chain = {'output' : generator_chain} | RunnableLambda(output_puller)  ## GIVEN

## END TODO
#####################################################################

rag_chain = retrieval_chain | generator_chain

# pprint(rag_chain.invoke("Tell me something interesting!"))
for token in rag_chain.stream("Tell me something interesting!"):
    print(token, end="")

Here's something interesting: Did you know that there are innovative methods being used to enhance document retrieval in various industries? For instance, in healthcare, a technique called "cascading document structures" is being used to improve retrieval-augmented generation (RAG) in medical guides and legal texts. This approach has proven advantageous and has drawn interest from leading RAG projects (HiQA: A Hierarchical Contextual Augmentation RAG for Multi-Documents QA).

<br>

### **Step 3:** Generating Synthetic Question-Answer Pairs

In this section, we can implement the first few part of our evaluation routine:

- **Sample the RAG agent document pool to find two document chunks.**
- **Use those two document chunks to generate a synthetic "baseline" question-answer pair.**
- Use the RAG agent to generate its own answer.
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [6]:
import random

num_questions = 3
synth_questions = []
synth_answers = []

simple_prompt = ChatPromptTemplate.from_messages([('system', '{system}'), ('user', 'INPUT: {input}')])

for i in range(num_questions):
    doc1, doc2 = random.sample(docs, 2)
    sys_msg = (
        "Use the documents provided by the user to generate an interesting question-answer pair."
        " Try to use both documents if possible, and rely more on the document bodies than the summary."
        " Use the format:\nQuestion: (good question, 1-3 sentences, detailed)\n\nAnswer: (answer derived from the documents)"
        " DO NOT SAY: \"Here is an interesting question pair\" or similar. FOLLOW FORMAT!"
    )
    usr_msg = (
        f"Document1: {format_chunk(doc1)}\n\n"
        f"Document2: {format_chunk(doc2)}"
    )

    qa_pair = (simple_prompt | llm).invoke({'system': sys_msg, 'input': usr_msg})
    synth_questions += [qa_pair.split('\n\n')[0]]
    synth_answers += [qa_pair.split('\n\n')[1]]
    pprint2(f"QA Pair {i+1}")
    pprint2(synth_questions[-1])
    pprint(synth_answers[-1])
    print()

QA Pair 1

Question: How can generative AI in the construction industry, as discussed in the first paper, effectively utilize 
the capabilities of Retrieval-Augmented Generation (RAG) to enhance productivity, quality, and safety?

Answer: By incorporating RAG techniques, as explored in the second paper, generative AI in the construction 
industry can benefit from the strengths of RAG, such as improved accuracy, credibility, and knowledge updates, to 
create more realistic and relevant data or content. This synergy between RAG and generative AI can enhance the 
productivity, quality, and safety of construction processes, as proposed in the first paper, by integrating 
domain-specific information and allowing for continuous learning and adaptation to the user's requirements.

QA Pair 2

Question: How can retrieval-augmented generation (RAG) be used to improve the performance and credibility of large 
language models (LLMs) in the construction industry, as seen in the development of generative models for querying 
contract documents?

Answer: According to the paper "Generative AI in the Construction Industry: A State-of-the-art Analysis", RAG 
improves the baseline LLM by 5.2, 9.4, and 4.8% in terms of quality, relevance, and reproducibility, respectively, 
providing a comprehensive analysis and practical framework to guide the adoption of generative AI techniques in the
construction industry. This is in line with the paper "Retrieval-Augmented Generation for Large Language Models: A 
Survey", which discusses the advancements in RAG systems and potential applications of RAG in various domains, 
including the construction industry.

QA Pair 3

Question: How do the authors of the Transformer model in Document 1 plan to extend and apply attention-based models
to different tasks and modalities?

Answer: The authors plan to extend the Transformer to problems involving input and output modalities other than 
text, such as images, audio, and video. They also plan to investigate local, restricted attention mechanisms to 
efficiently handle large inputs and outputs.

<br>

### **Step 4:** Answer The Synthetic Questions

In this section, we can implement the third part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- **Use the RAG agent to generate its own answer.**
- Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [7]:
## TODO: Generate some synthetic answers to the questions above.
##   Try to use the same syntax as the cell above
rag_answers = []
for i, q in enumerate(synth_questions):
    ## TODO: Compute the RAG Answer
    #rag_answer = ""
    rag_answer = rag_chain.invoke(q)
    rag_answers += [rag_answer]
    pprint2(f"QA Pair {i+1}", q, "", sep="\n")
    pprint(f"RAG Answer: {rag_answer}", "", sep='\n')

QA Pair 1
Question: How can generative AI in the construction industry, as discussed in the first paper, effectively utilize 
the capabilities of Retrieval-Augmented Generation (RAG) to enhance productivity, quality, and safety?

RAG Answer: The first paper discusses the potential of generative AI in the construction industry, and one of the 
notable applications mentioned is the use of Retrieval-Augmented Generation (RAG) to enhance productivity, quality,
and safety. In fact, the case study presented in the paper highlights the effectiveness of using RAG to improve the
ability to extract accurate and relevant information from construction contract documents.

According to the paper, the RAG system outperformed the baseline generative model (GPT-4) by 5.2% in terms of 
quality, 9.4% in terms of relevance, and 4.8% in terms of reproducibility. This suggests that RAG can be a valuable
tool in the construction industry, particularly when it comes to information retrieval and contract management.

To put this into practice, construction firms can leverage RAG to improve their workflows and decision-making 
processes. For example, they can use RAG to extract relevant information from contract documents more accurately 
and efficiently, which can help reduce errors and improve productivity. Additionally, RAG can be used to identify 
potential safety hazards and mitigate risks more effectively, which can contribute to improved safety outcomes.

Overall, the study provides valuable insights into the application of RAG in the construction industry, and 
highlights its potential to improve productivity, quality, and safety.

QA Pair 2
Question: How can retrieval-augmented generation (RAG) be used to improve the performance and credibility of large 
language models (LLMs) in the construction industry, as seen in the development of generative models for querying 
contract documents?

RAG Answer: RAG can be a game-changer for LLMs in the construction industry. According to our research, RAG 
integrates semantic search over a domain-specific knowledge base into the text generation pipeline, allowing you to
retrieve the most relevant contextual examples from the contract text to prime the LLM when responding to queries 
[1].

Think of it like this: when a query is made, the RAG framework helps ground the model's outputs in the actual 
contract content, reducing the likelihood of "hallucinations" or factually incorrect content. This process enhances
the model's performance and credibility by making it more accurate and relevant.

As demonstrated in a case study on applying generative models for enhanced querying of construction contract 
documents, the RAG system showed a significantly improved ability to extract accurate, relevant information from 
contracts [1]. In fact, the results showed that RAG improved the baseline LLM by 5.2%, 9.4%, and 4.8% in terms of 
quality, relevance, and reproducibility, respectively.

By leveraging RAG, construction firms can build customized generative AI solutions that are more effective and 
trustworthy. The integration of RAG into LLMs like GPT-4, a proprietary model designed by OpenAI, has been 
particularly successful in this realm.

Do you have any follow-up questions or would you like more information on how RAG can be applied in the 
construction industry?

QA Pair 3
Question: How do the authors of the Transformer model in Document 1 plan to extend and apply attention-based models
to different tasks and modalities?

RAG Answer: So, according to the document, the authors are quite excited about the future of attention-based 
models, and they plan to apply them to various tasks. They mention that they're going to extend the Transformer to 
problems involving input and output modalities other than text, such as images, audio, and video. They also plan to
investigate local, restricted attention mechanisms to handle large inputs and outputs efficiently. Additionally, 
they're looking to make generation less sequential, which is another research goal of theirs.

Interestingly, they mention that they want to explore tasks where the output is not necessarily a sequence of 
symbols, like in text generation. And, they're planning to release their code to make it easier for others to build
upon their work.

<br>

### **Step 5:** Implement A Human Preference Metric

In this section, we can implement the fourth part of our evaluation routine:

- Sample the RAG agent document pool to find two document chunks.
- Use those two document chunks to generate a synthetic "baseline" question-answer pair.
- Use the RAG agent to generate its own answer.
- **Use a judge LLM to compare the two responses while grounding the synthetic generation as "ground-truth correct."**

The chain should be a simple but powerful process that tests for the following objective:

> Does my RAG chain outperform a narrow chatbot with limited document access?

In [13]:
## TODO: Adapt this prompt for whichever LLM you're actually interested in using. 
## If it's llama, maybe system message would be good?
eval_prompt = ChatPromptTemplate.from_template("""INSTRUCTION: 
Evaluate the following Question-Answer pair for human preference and consistency.
Assume the first answer is a ground truth answer and has to be correct.
Assume the second answer may or may not be true.
[1] The second answer lies, does not answer the question, or is inferior to the first answer.
[2] The second answer is better than the first and does not introduce any inconsistencies.

Output Format:
[Score] Justification

{qa_trio}

EVALUATION: 
""")

pref_score = []

trio_gen = zip(synth_questions, synth_answers, rag_answers)
for i, (q, a_synth, a_rag) in enumerate(trio_gen):
    pprint2(f"Set {i+1}\n\nQuestion: {q}\n\n")

    qa_trio = f"Question: {q}\n\nAnswer 1 (Ground Truth): {a_synth}\n\n Answer 2 (New Answer): {a_rag}"
    pref_score += [(eval_prompt | llm).invoke({'qa_trio': qa_trio})]
    pprint(f"Synth Answer: {a_synth}\n\n")
    pprint(f"RAG Answer: {a_rag}\n\n")
    pprint2(f"Synth Evaluation: {pref_score[-1]}\n\n")

Set 1

Question: Question: How can generative AI in the construction industry, as discussed in the first paper, 
effectively utilize the capabilities of Retrieval-Augmented Generation (RAG) to enhance productivity, quality, and 
safety?

Synth Answer: Answer: By incorporating RAG techniques, as explored in the second paper, generative AI in the 
construction industry can benefit from the strengths of RAG, such as improved accuracy, credibility, and knowledge 
updates, to create more realistic and relevant data or content. This synergy between RAG and generative AI can 
enhance the productivity, quality, and safety of construction processes, as proposed in the first paper, by 
integrating domain-specific information and allowing for continuous learning and adaptation to the user's 
requirements.

RAG Answer: The first paper discusses the potential of generative AI in the construction industry, and one of the 
notable applications mentioned is the use of Retrieval-Augmented Generation (RAG) to enhance productivity, quality,
and safety. In fact, the case study presented in the paper highlights the effectiveness of using RAG to improve the
ability to extract accurate and relevant information from construction contract documents.

According to the paper, the RAG system outperformed the baseline generative model (GPT-4) by 5.2% in terms of 
quality, 9.4% in terms of relevance, and 4.8% in terms of reproducibility. This suggests that RAG can be a valuable
tool in the construction industry, particularly when it comes to information retrieval and contract management.

To put this into practice, construction firms can leverage RAG to improve their workflows and decision-making 
processes. For example, they can use RAG to extract relevant information from contract documents more accurately 
and efficiently, which can help reduce errors and improve productivity. Additionally, RAG can be used to identify 
potential safety hazards and mitigate risks more effectively, which can contribute to improved safety outcomes.

Overall, the study provides valuable insights into the application of RAG in the construction industry, and 
highlights its potential to improve productivity, quality, and safety.

Synth Evaluation: [Score] 2 Justification

The second answer is clearly better than the first and does not introduce any inconsistencies. Here's why:

1. **More specific and actionable**: The second answer provides concrete examples and case studies that demonstrate
the effectiveness of RAG in the construction industry. It is more specific and actionable than the first answer, 
which is more general and does not provide concrete evidence.
2. **Provides more value**: The second answer provides more value by highlighting the actual performance and 
outcomes of using RAG in the construction industry, specifically the percentage improvements in quality, relevance,
and reproducibility. This adds depth and credibility to the answer.
3. **More informative**: The second answer is more informative, as it explicitly mentions the case study, the RAG 
system's performance, and potential applications of RAG in the construction industry. This provides more context 
and helps readers understand the relevance and impact of RAG in the industry.
4. **No inconsistencies**: Both answers are consistent with each other, as they both discuss the potential of 
generative AI in the construction industry and the effectiveness of RAG in enhancing productivity, quality, and 
safety. However, the second answer provides a more nuanced and detailed discussion that supports its claims with 
concrete evidence.

Overall, the second answer is scores higher due to its more specific, actionable, and informative content that 
provides valuable insights into the application of RAG in the construction industry.

Set 2

Question: Question: How can retrieval-augmented generation (RAG) be used to improve the performance and credibility
of large language models (LLMs) in the construction industry, as seen in the development of generative models for 
querying contract documents?

...

Synth Evaluation: [Score] 1 Justification

The second answer introduces two inconsistencies with the first answer:

1. In the second answer, it mentions "according to our research", whereas in the first answer, the improvements in 
LLMs are attributed to the papers "Generative AI in the Construction Industry: A State-of-the-art Analysis" and 
"Retrieval-Augmented Generation for Large Language Models: A Survey". This inconsistency in citation suggests that 
the second answer is presenting new or unpublished results that may not be reliable.

2. The second answer mentions "a case study on applying generative models for enhanced querying of construction 
contract documents", but the improvements in LLMs (5.2%, 9.4%, and 4.8%) are identical to those mentioned in the 
first answer, indicating a possible copy-paste error or attempt to support the first answer with new results. 
However, the fact that the second answer does not provide any new or different information, yet claims to be a 
different answer, suggests a lack of originality and makes the second answer inferior to the first.

Furthermore, while the second answer is well-written and attempts to justify the use of RAG in LLMs, it fails to 
introduce new insights or information that would make it better than the first answer. Hence, the second answer is 
scored 1, indicating it is worse than the first answer due to the inconsistencies and lack of originality.

Set 3

Question: Question: How do the authors of the Transformer model in Document 1 plan to extend and apply 
attention-based models to different tasks and modalities?

Synth Answer: Answer: The authors plan to extend the Transformer to problems involving input and output modalities 
other than text, such as images, audio, and video. They also plan to investigate local, restricted attention 
mechanisms to efficiently handle large inputs and outputs.

RAG Answer: So, according to the document, the authors are quite excited about the future of attention-based 
models, and they plan to apply them to various tasks. They mention that they're going to extend the Transformer to 
problems involving input and output modalities other than text, such as images, audio, and video. They also plan to
investigate local, restricted attention mechanisms to handle large inputs and outputs efficiently. Additionally, 
they're looking to make generation less sequential, which is another research goal of theirs.

Interestingly, they mention that they want to explore tasks where the output is not necessarily a sequence of 
symbols, like in text generation. And, they're planning to release their code to make it easier for others to build
upon their work.

Synth Evaluation: [Score] 1 Justification

The second answer introduces no new information that contradicts the ground truth answer, but it also does not 
provide any new insights or details not mentioned in the first answer. However, the format of the second answer is 
more verbose and repetitive, making it less conducive to a clear and concise response.

While the second answer mentions "less sequential" generation, which is a new concept, it does not specify how this
would be achieved, which makes it harder to determine if this is a valid addition to the first answer. Moreover, 
the claim about releasing code does not relate to the question asked about extending and applying attention-based 
models.

Therefore, the score remains 1, indicating that the second answer is similar to the first but does not add any new 
information that is not explicitly contradicted.

<br>

**Congratulations! We now have an LLM system that reasons about our pipeline and tries to evaluate it!** Now that we have some judge results, we can simply aggregate the results and see how often our formulation was according to an LLM:

In [12]:
pref_score = sum(("[2]" in score) for score in pref_score) / len(pref_score)
print(f"Preference Score: {pref_score}")

Preference Score: 1.0


----

<br>

## **Part 4:** Advanced Formulations

The exercise above was meant to prepare you for the final assessment of the course and showcased a simple but effective evaluator chain. The objective and implementation details were provided for you, and the logic for using it probably makes sense now that you've seen it in action. 

With that being said, this metric was merely a product of us specifying:
- **What kind of behavior is important for our pipeline to have?**
- **What do we need to do in order to exhibit and evaluate this behavior?**

From these two questions, we could have come up with plenty of other evaluation metrics that could have assessed different attributes, incorporated different evaluator chain techniques, and even required different pipeline organization strategies. Though far from an exhaustive list, some common formulations you will likely come across may include:

- **Style Evaluation:** Some evaluation formulations can be as simple as "let me ask some questions and see if the output feels desirable." This might be used to see whether a chatbot "acts like it's supposed to" based on a description provided to a judge LLM. We're using quotations since this kind of assessment can reasonably be achieved with nothing but prompt engineering and a while loop.

- **Ground-Truth Evaluation:** In our chain, we used synthetic generation to create some random questions and answers using a sampling strategy, but in reality you may actually have some representative questions and answers that you need your chatbot to consistently get right! In this case, a modification of the exercise chain above should be implemented and closely monitored as you develop your pipelines.

- **Retrieval/Augmentation Evaluation:** This course made many assumptions about what kinds of preprocessing and prompting steps would be good for your pipelines, and much of this was determined by experimentation. Factors such as document preprocessing, chunking strategies, model selection, and prompt specification all played important roles, so creating metrics to validate these decisions may be of interest. This kind of metric might require your pipeline to output your context chunks or may even rely solely on embedding similarity comparisons, so keep this in mind when trying to implement a chain that works with multiple evaluation strategies. Consider the [**RagasEvaluatorChain**](https://docs.ragas.io/en/stable/howtos/integrations/langchain.html) abstraction as a decent starting point for making an custom generalizable evaluation routine. 

- **Trajectory Evaluation:** Using more advanced agent formulations, you can implement multiple-query strategies that assume the presence of conversational memory. With this, you can implement an evaluation agent which can:
    - Ask a series of questions in order to evaluate how well the agent is able to adapt and cater to the scenario. This kind of system generally considers a series of correspondence and aims to tease out and evaluate a "trajectory" of how the agent navigated the conversation. The [**LangChain Trajectory Evaluators documentation**](https://python.langchain.com/v0.1/docs/guides/productionization/evaluation/trajectory/) is a good starting point.
    - Alternatively, you could also implement an evaluation agent that tries to achieve objectives by interacting with the chatbot. Such an agent can output whether they were able to navigate to their solution in a natural manner, and can even be used to generate a report about the percieved performance. The [**LangChain Agents documentation**](https://python.langchain.com/v0.1/docs/modules/agents/) is a good starting point!

<br>

At the end of the day, just make sure to use the tools you have at your disposal appropriately. By this point in the course, you should already be well-acquainted with the LLM core value propositions: **They're powerful, scalable, predictable, controllable, and orchestratable... but will act unpredictably when you just expect them to work by default.** Assess your needs, formulate and validate your pipelines, give enough information, and add as much control as you can to make your system work consistently, efficiently, and effectively.

----

<br>

## **Part 5: [Assessment]** Evaluating For Credit

Welcome to the last exercise of the course! Hopefully you've enjoyed the material and are ready to actually get credit for these notebooks! For this part:

- **Make sure you're in the course environment**
- **Make sure `docstore_index/` has been uploaded to the course environment...**
    - **...and contains [at least one Arxiv paper](https://arxiv.org/search/advanced) which has been updated recently.**
- **Make sure you don't have some old session of [`09_langserve.ipynb`](09_langserve.ipynb) already occupying the port. Your assessment requires you to implement the new `/retriever` and `/generator` endpoints!!**

**Objective:** On launch, [**`frontend/frontend_block.py`**](frontend/frontend_block.py) had several lines of code which trigger the course pass condition. Your objective is to invoke that series of commands by using your pipeline to pass the **Evaluation** check! Recall [`09_langserve.ipynb`](09_langserve.ipynb) and use it as a starting example! As a recommendation, consider duplicating it so that you can keep the original as an authoritative reference. 

**Once Finished:** While your course environment is still open, please navigate back to your course environment launcher area and click the **"Assess Task"** button! After that, you're all done!

In [ ]:
%%js
var url = 'http://'+window.location.host+':8090';
element.innerHTML = '<a style="color:green;" target="_blank" href='+url+'><h1>< Link To Gradio Frontend ></h1></a>';

----

<br>

## <font color="#76b900">**Congratulations On Completing The Course**</font>

Hopefully this course was not only exciting and challenging, but also adequately prepared you for work on the cutting edge of LLM and RAG system development! Going forward, you should have the skills necessary to tackle industry-level challenges and explore RAG deployment with open-source models and frameworks.

**Some NVIDIA-specific releases related to this that you may find interesting include:**
- [**NVIDIA NIM**](https://www.nvidia.com/en-us/ai/), which offers microservice spinup routines that can be deployed on local compute.
- [**TensorRT-LLM**](https://github.com/NVIDIA/TensorRT-LLM) is the current recommended framework for deploying GPU-accelerated LLM model engines in production settings.
- [**NVIDIA's Generative AI Examples Repo**](https://github.com/NVIDIA/GenerativeAIExamples), which includes the current canonical microservice example application and will be updated with new resources as new production workflows get released.
- [**The Knowledge-Based Chatbot Technical Brief**](https://resources.nvidia.com/en-us-generative-ai-chatbot-workflow/knowledge-base-chatbot-technical-brief) which discusses additional publicly-accessible details on productionalizing RAG systems.

**Additionally, some key topics you may be interested in delving more into include:**
- [**LlamaIndex**](https://www.llamaindex.ai/), which has strong components that can augment and occasionally improve upon the LangChain RAG features.
- [**LangSmith**](https://docs.smith.langchain.com/), an upcoming agent productionalization service offered by LangChain.
- [**Gradio**](https://www.gradio.app/), though touched on in the course, has many more interface options which will be worth investigating. For inspiration, consider checking out [**HuggingFace Spaces**](https://huggingface.co/spaces) for examples.
- [**LangGraph**](https://python.langchain.com/docs/langgraph/) is a framework for graph-based LLM orchestration, and is a natural next step forward for those interested in [multi-agent workflows](https://blog.langchain.dev/langgraph-multi-agent-workflows/).
- [**DSPy**](https://github.com/stanfordnlp/dspy), a flow engineering framework that allows you to optimize LLM orchestration pipelines based on empirical performance results.

<center><a href="https://www.nvidia.com/en-us/training/"><img src="https://dli-lms.s3.amazonaws.com/assets/general/DLI_Header_White.png" width="400" height="186" /></a></center>